In [1]:
import ROOT
import pandas as pd
import numpy as np 

import statistics
from statistics import mean
# Use: mean(liste)
from itertools import combinations
from collections import Counter

def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]

import os
import sys

import matplotlib as mpl

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
mpl.rcParams['image.cmap'] = 'hot_r'

from matplotlib import rc
url = 'https://gist.githubusercontent.com/bonanomi/d14780f7562cb2a22fdd753a9d4459d4/raw/034716767493fcfb7852c0c0e4555b86eafbb901/MyMPLStyle'
plt.style.use(url)
rc('font',**{'family':'DejaVu Sans'})

Welcome to JupyROOT 6.14/04


In [2]:
indir = '/data_cms_upgrade/motta/HGCAL_SKIMS/SKIM_20Apr2021'
rfile = ROOT.TFile.Open(indir+'/SKIM_RelValTenTau_noPU_caloTruth/mergedOutput.root')
tree = rfile.Get('HGCALskimmedTree')

#OLD FILES
#indir = '/data_CMS_upgrade/motta/HGCAL_SKIMS/SKIM_31Mar2021'
#rfile = ROOT.TFile.Open(indir+'/SKIM_RelValTenTau_noPU/mergedOutput.root')
#tree = rfile.Get('SkimmedTree')

#print(tree.GetEntries())
#tree.Show(41)

# Hard Selection : we remove event if DeltaR between taus < 0.1

In [3]:
#This is working, we have indeed 58 events that return a False in at least one endcap, 5 events with false for the two endcaps
#We select taus that are at leat DeltaR = 2 apart
def taus_selection_hard(taus_endcap, gentau_vis_eta, gentau_vis_phi):
    
    Delta_R = 0
    
    for i,j in combinations(taus_endcap, 2): #to avoid repetitions and (n,n) cases (same tau)
            Delta_R = np.sqrt((tree.gentau_vis_eta[int(i)]-tree.gentau_vis_eta[int(j)])**2
            +(tree.gentau_vis_phi[int(i)]- tree.gentau_vis_phi[int(j)])**2)  

            if Delta_R < 0.1:
                return False
    return True

# We match cluster and tau if deltaR_cluster_tau < 1 and taking the cluster with the min DeltaR_cluster_tau

In [4]:
#This will return (tau index, closest cluster index) for selected taus of one event
def match_closest_Delta_R(taus_endcap, tau_clt_matching, cl3d_eta, cl3d_phi):
    cpt = 0#
    indices_used = []
    
    if len(taus_endcap) > 0 :
        for i in taus_endcap :
            Delta_R_taus_clusters = []
            if len(tree.cl3d_eta) > 0 :
                for j in range (0,len(tree.cl3d_eta)) :
                    Delta_R_taus_clusters.append(np.sqrt((tree.gentau_vis_eta[i]-tree.cl3d_eta[j])**2
                                +(tree.gentau_vis_phi[i]- tree.cl3d_phi[j])**2))
                    
                if min(Delta_R_taus_clusters) < 1 :
                    min_index = Delta_R_taus_clusters.index(min(Delta_R_taus_clusters))
                    if min_index not in indices_used :#we don't want to associate a cluster to many taus
                        indices_used.append(min_index)
                        tau_clt_matching.append([i,min_index])  
                if min(Delta_R_taus_clusters) > 1 :#
                    cpt += 1#
            if len(tree.cl3d_eta) == 0 : #this is the anomaly
                cpt == len(taus_endcap) #we loose the event
                break
    if cpt == len(taus_endcap):#
        booleen = True
    else:#
        booleen = False
    return (tau_clt_matching,booleen)

# Main function looping on the events

In [5]:
#Variables to check how many events are lost
no_tau = 0
Tau_toclose_AND_tau_clt_tofaraway = 0
tau_toclose_in_endcap1_AND_endcap2 = 0
cl3d_eta_0 = 0
taus_and_clt_tofaraway_in_endcap1_and_endcap2 = 0

all_events = []
events_endcap1 = []
events_endcap2 = []
tau_cl3d_matched_endcap1 = []
tau_cl3d_matched_endcap2 = []

for evt in range(tree.GetEntries()):
    
    tree.GetEntry(evt)
    all_events.append(evt)
    l = tree.gentau_n #nb of tau in the event

    taus_endcap1 =[]
    taus_endcap2 =[]
    
    taus_toclose =[[],[]]
    taus_toclose_temp = [[],[]]
    
    Delta_R = 0
    
    #To check and count
    if l == 0 :
        no_tau += 1
    #To check and count
    if len(tree.cl3d_eta) == 0 and l > 0:
        cl3d_eta_0 += 1
    
    for i in range(0,l):
        if tree.gentau_vis_eta[i] > 0 :
            taus_endcap1.append(i)
        if tree.gentau_vis_eta[i] < 0 :
            taus_endcap2.append(i)

    gentau_vis_eta = tree.gentau_vis_eta
    gentau_vis_phi = tree.gentau_vis_phi

    cl3d_eta = tree.cl3d_eta
    cl3d_phi = tree.cl3d_phi
    
    #HARD SELECTION : if one deltaR>2, we threw away the event
    selected_endcap1 = taus_selection_hard(taus_endcap1, gentau_vis_eta, gentau_vis_phi)
    selected_endcap2 = taus_selection_hard(taus_endcap2, gentau_vis_eta, gentau_vis_phi)
    
    #To check and count
    if selected_endcap1 == True and selected_endcap2 == True :
        tau_clt_matching_temp = []
        variable1,variable2 = match_closest_Delta_R(taus_endcap1, tau_clt_matching_temp, cl3d_eta, cl3d_phi)
        variable11,variable22 = match_closest_Delta_R(taus_endcap2, tau_clt_matching_temp, cl3d_eta, cl3d_phi)
        if variable2 == True and variable22 == True :
            taus_and_clt_tofaraway_in_endcap1_and_endcap2 += 1
    
    #To check and count
    if selected_endcap1 == False and selected_endcap2 == False :
        tau_toclose_in_endcap1_AND_endcap2 += 1
    
    if  selected_endcap1 == True :
        tau_clt_matching_temp = []
        tau_clt_matching, booleen1  = match_closest_Delta_R(taus_endcap1, tau_clt_matching_temp, cl3d_eta, cl3d_phi)
        
        #To check and count
        if selected_endcap2 == False and booleen1 == True :
            Tau_toclose_AND_tau_clt_tofaraway += 1
        
        if len(tau_clt_matching) > 0 :
            tau_cl3d_matched_endcap1.append(tau_clt_matching)
            events_endcap1.append(evt)
    
    if  selected_endcap2 == True :
        tau_clt_matching_temp = []
        tau_clt_matching, booleen2  = match_closest_Delta_R(taus_endcap2, tau_clt_matching_temp, cl3d_eta, cl3d_phi)
        
        #To check and count
        if selected_endcap1 == False and booleen2 == True :
            Tau_toclose_AND_tau_clt_tofaraway += 1
        
        if len(tau_clt_matching) > 0 :
            tau_cl3d_matched_endcap2.append(tau_clt_matching)
            events_endcap2.append(evt)

events_selected = list(set(events_endcap1) | set(events_endcap2))
events_rejected = [elem for elem in all_events if elem not in events_selected]

In [6]:
print('Number of events with :')
print('no taus : %i' %(no_tau))
print('no taus too close in one endcap, and taus and clusters to far away in the other endcap : %i' %(Tau_toclose_AND_tau_clt_tofaraway))
print('taus too close in endcap1 and in endcap2 : %i' %(tau_toclose_in_endcap1_AND_endcap2))
print('len(tree.cl3d_eta) = 0 : %i' %(cl3d_eta_0))
print('taus and clusters too far away in endcap1 and endcap2 : %i ' %(taus_and_clt_tofaraway_in_endcap1_and_endcap2 - no_tau))
print('\n')
print('We should have %i events selected' %(len(all_events) - no_tau - Tau_toclose_AND_tau_clt_tofaraway - tau_toclose_in_endcap1_AND_endcap2 - cl3d_eta_0 - (taus_and_clt_tofaraway_in_endcap1_and_endcap2 - no_tau)))
print('--> and we have %i events selected ' %(len(set(events_endcap1) | set(events_endcap2))))

Number of events with :
no taus : 0
no taus too close in one endcap, and taus and clusters to far away in the other endcap : 7
taus too close in endcap1 and in endcap2 : 5
len(tree.cl3d_eta) = 0 : 188
taus and clusters too far away in endcap1 and endcap2 : 151 


We should have 8120 events selected
--> and we have 8120 events selected 


In [7]:
print('This is endcap1')
for i in range(0,len(events_endcap1)):
    tree.GetEntry(events_endcap1[i])
    nb_tau = len(tau_cl3d_matched_endcap1[i])
    print('This is the event number %u corresponding to the entry %i with %i tau(s) selected' %(events_endcap1[i], tree.event,nb_tau))
    
    for p in range(0,nb_tau):
        tau_index = tau_cl3d_matched_endcap1[i][p][0]
        cl3d_index = tau_cl3d_matched_endcap1[i][p][1]
        Delta_R_to_print = np.sqrt((tree.gentau_vis_eta[tau_index]-tree.cl3d_eta[cl3d_index])**2
                        +(tree.gentau_vis_phi[tau_index]- tree.cl3d_phi[cl3d_index])**2)
        print('Tau index = %i - Cluster index = %i - Delta_R = %f' %(tau_index, cl3d_index, Delta_R_to_print))
    print('\n')

This is endcap1
This is the event number 1 corresponding to the entry 7302 with 2 tau(s) selected
Tau index = 2 - Cluster index = 2 - Delta_R = 0.020547
Tau index = 4 - Cluster index = 1 - Delta_R = 0.080311


This is the event number 2 corresponding to the entry 7307 with 2 tau(s) selected
Tau index = 0 - Cluster index = 2 - Delta_R = 0.012565
Tau index = 1 - Cluster index = 1 - Delta_R = 0.013425


This is the event number 4 corresponding to the entry 7312 with 1 tau(s) selected
Tau index = 4 - Cluster index = 2 - Delta_R = 0.027327


This is the event number 5 corresponding to the entry 7305 with 1 tau(s) selected
Tau index = 0 - Cluster index = 3 - Delta_R = 0.016239


This is the event number 6 corresponding to the entry 7304 with 1 tau(s) selected
Tau index = 1 - Cluster index = 1 - Delta_R = 0.031314


This is the event number 8 corresponding to the entry 7315 with 1 tau(s) selected
Tau index = 0 - Cluster index = 3 - Delta_R = 0.049816


This is the event number 9 corresponding

This is the event number 358 corresponding to the entry 7786 with 2 tau(s) selected
Tau index = 0 - Cluster index = 2 - Delta_R = 0.042999
Tau index = 2 - Cluster index = 1 - Delta_R = 0.026481


This is the event number 359 corresponding to the entry 7785 with 3 tau(s) selected
Tau index = 2 - Cluster index = 9 - Delta_R = 0.004907
Tau index = 4 - Cluster index = 11 - Delta_R = 0.010374
Tau index = 5 - Cluster index = 6 - Delta_R = 0.023394


This is the event number 360 corresponding to the entry 7775 with 1 tau(s) selected
Tau index = 0 - Cluster index = 1 - Delta_R = 0.012143


This is the event number 361 corresponding to the entry 7773 with 2 tau(s) selected
Tau index = 1 - Cluster index = 3 - Delta_R = 0.020176
Tau index = 2 - Cluster index = 4 - Delta_R = 0.020264


This is the event number 362 corresponding to the entry 7783 with 1 tau(s) selected
Tau index = 1 - Cluster index = 2 - Delta_R = 0.011103


This is the event number 363 corresponding to the entry 7774 with 3 tau(s)

Tau index = 0 - Cluster index = 1 - Delta_R = 0.023262


This is the event number 565 corresponding to the entry 8006 with 1 tau(s) selected
Tau index = 1 - Cluster index = 3 - Delta_R = 0.024637


This is the event number 566 corresponding to the entry 8009 with 1 tau(s) selected
Tau index = 3 - Cluster index = 2 - Delta_R = 0.010540


This is the event number 567 corresponding to the entry 8005 with 3 tau(s) selected
Tau index = 0 - Cluster index = 1 - Delta_R = 0.047423
Tau index = 1 - Cluster index = 0 - Delta_R = 0.048594
Tau index = 3 - Cluster index = 2 - Delta_R = 0.024110


This is the event number 569 corresponding to the entry 8004 with 1 tau(s) selected
Tau index = 0 - Cluster index = 1 - Delta_R = 0.020018


This is the event number 572 corresponding to the entry 8014 with 2 tau(s) selected
Tau index = 1 - Cluster index = 2 - Delta_R = 0.015307
Tau index = 2 - Cluster index = 4 - Delta_R = 0.018699


This is the event number 573 corresponding to the entry 8012 with 1 tau(s

Tau index = 0 - Cluster index = 3 - Delta_R = 0.041419
Tau index = 2 - Cluster index = 4 - Delta_R = 0.037102


This is the event number 709 corresponding to the entry 8159 with 1 tau(s) selected
Tau index = 0 - Cluster index = 1 - Delta_R = 0.023428


This is the event number 710 corresponding to the entry 8155 with 3 tau(s) selected
Tau index = 0 - Cluster index = 4 - Delta_R = 0.010871
Tau index = 1 - Cluster index = 5 - Delta_R = 0.009470
Tau index = 3 - Cluster index = 3 - Delta_R = 0.006002


This is the event number 711 corresponding to the entry 8163 with 1 tau(s) selected
Tau index = 1 - Cluster index = 1 - Delta_R = 0.009460


This is the event number 712 corresponding to the entry 8165 with 2 tau(s) selected
Tau index = 0 - Cluster index = 2 - Delta_R = 0.011756
Tau index = 2 - Cluster index = 4 - Delta_R = 0.017307


This is the event number 713 corresponding to the entry 8160 with 3 tau(s) selected
Tau index = 0 - Cluster index = 2 - Delta_R = 0.021135
Tau index = 1 - Clus

Tau index = 0 - Cluster index = 9 - Delta_R = 0.048807
Tau index = 2 - Cluster index = 7 - Delta_R = 0.066471
Tau index = 4 - Cluster index = 8 - Delta_R = 0.022426


This is the event number 914 corresponding to the entry 8482 with 1 tau(s) selected
Tau index = 1 - Cluster index = 2 - Delta_R = 0.005940


This is the event number 916 corresponding to the entry 8480 with 2 tau(s) selected
Tau index = 1 - Cluster index = 2 - Delta_R = 0.018878
Tau index = 2 - Cluster index = 1 - Delta_R = 0.016362


This is the event number 917 corresponding to the entry 8486 with 2 tau(s) selected
Tau index = 2 - Cluster index = 3 - Delta_R = 0.013482
Tau index = 3 - Cluster index = 4 - Delta_R = 0.007834


This is the event number 918 corresponding to the entry 8490 with 2 tau(s) selected
Tau index = 0 - Cluster index = 5 - Delta_R = 0.034519
Tau index = 1 - Cluster index = 3 - Delta_R = 0.754514


This is the event number 919 corresponding to the entry 8489 with 3 tau(s) selected
Tau index = 0 - Clus

...

This is the event number 7716 corresponding to the entry 1199 with 2 tau(s) selected
Tau index = 0 - Cluster index = 1 - Delta_R = 0.014174
Tau index = 1 - Cluster index = 2 - Delta_R = 0.011103


This is the event number 7717 corresponding to the entry 1200 with 1 tau(s) selected
Tau index = 1 - Cluster index = 2 - Delta_R = 0.016473


This is the event number 7718 corresponding to the entry 1197 with 3 tau(s) selected
Tau index = 1 - Cluster index = 5 - Delta_R = 0.011918
Tau index = 3 - Cluster index = 4 - Delta_R = 0.006087
Tau index = 5 - Cluster index = 3 - Delta_R = 0.038395


This is the event number 7719 corresponding to the entry 1309 with 2 tau(s) selected
Tau index = 0 - Cluster index = 2 - Delta_R = 0.019238
Tau index = 3 - Cluster index = 3 - Delta_R = 0.029827


This is the event number 7720 corresponding to the entry 1303 with 2 tau(s) selected
Tau index = 1 - Cluster index = 2 - Delta_R = 0.047845
Tau index = 3 - Cluster index = 3 - Delta_R = 0.018477


This is the eve

This is the event number 7938 corresponding to the entry 2827 with 3 tau(s) selected
Tau index = 0 - Cluster index = 6 - Delta_R = 0.027463
Tau index = 3 - Cluster index = 4 - Delta_R = 0.040045
Tau index = 5 - Cluster index = 5 - Delta_R = 0.041821


This is the event number 7939 corresponding to the entry 2829 with 1 tau(s) selected
Tau index = 0 - Cluster index = 4 - Delta_R = 0.005157


This is the event number 7940 corresponding to the entry 2831 with 1 tau(s) selected
Tau index = 0 - Cluster index = 0 - Delta_R = 0.012242


This is the event number 7941 corresponding to the entry 2833 with 2 tau(s) selected
Tau index = 0 - Cluster index = 3 - Delta_R = 0.022074
Tau index = 3 - Cluster index = 2 - Delta_R = 0.019503


This is the event number 7942 corresponding to the entry 2832 with 3 tau(s) selected
Tau index = 0 - Cluster index = 6 - Delta_R = 0.006553
Tau index = 3 - Cluster index = 7 - Delta_R = 0.014177
Tau index = 6 - Cluster index = 4 - Delta_R = 0.014861


This is the eve

This is the event number 8062 corresponding to the entry 3170 with 1 tau(s) selected
Tau index = 0 - Cluster index = 0 - Delta_R = 0.007110


This is the event number 8063 corresponding to the entry 3166 with 3 tau(s) selected
Tau index = 1 - Cluster index = 3 - Delta_R = 0.011847
Tau index = 3 - Cluster index = 4 - Delta_R = 0.103668
Tau index = 5 - Cluster index = 2 - Delta_R = 0.015670


This is the event number 8064 corresponding to the entry 3165 with 3 tau(s) selected
Tau index = 1 - Cluster index = 7 - Delta_R = 0.043931
Tau index = 2 - Cluster index = 8 - Delta_R = 0.015987
Tau index = 3 - Cluster index = 5 - Delta_R = 0.054177


This is the event number 8065 corresponding to the entry 3168 with 1 tau(s) selected
Tau index = 0 - Cluster index = 2 - Delta_R = 0.014068


This is the event number 8066 corresponding to the entry 3171 with 1 tau(s) selected
Tau index = 1 - Cluster index = 1 - Delta_R = 0.017531


This is the event number 8067 corresponding to the entry 3169 with 2 t

Tau index = 3 - Cluster index = 3 - Delta_R = 0.018211


This is the event number 8291 corresponding to the entry 3506 with 1 tau(s) selected
Tau index = 1 - Cluster index = 3 - Delta_R = 0.025078


This is the event number 8294 corresponding to the entry 3501 with 2 tau(s) selected
Tau index = 0 - Cluster index = 6 - Delta_R = 0.015068
Tau index = 1 - Cluster index = 5 - Delta_R = 0.024609


This is the event number 8295 corresponding to the entry 3514 with 2 tau(s) selected
Tau index = 0 - Cluster index = 1 - Delta_R = 0.031148
Tau index = 1 - Cluster index = 2 - Delta_R = 0.013293


This is the event number 8296 corresponding to the entry 3503 with 1 tau(s) selected
Tau index = 3 - Cluster index = 5 - Delta_R = 0.016579


This is the event number 8297 corresponding to the entry 3516 with 1 tau(s) selected
Tau index = 0 - Cluster index = 1 - Delta_R = 0.566121


This is the event number 8298 corresponding to the entry 3517 with 2 tau(s) selected
Tau index = 1 - Cluster index = 3 - De

Tau index = 5 - Cluster index = 4 - Delta_R = 0.020056




In [8]:
print('This is endcap2')
for i in range(0,len(events_endcap2)):
    tree.GetEntry(events_endcap2[i])
    nb_tau = len(tau_cl3d_matched_endcap2[i])
    print('This is the event number %u corresponding to the entry %i with %i tau(s) selected' %(events_endcap2[i], tree.event,nb_tau))
    
    for p in range(0,nb_tau):
        tau_index = tau_cl3d_matched_endcap2[i][p][0]
        cl3d_index = tau_cl3d_matched_endcap2[i][p][1]
        Delta_R_to_print = np.sqrt((tree.gentau_vis_eta[tau_index]-tree.cl3d_eta[cl3d_index])**2
                        +(tree.gentau_vis_phi[tau_index]- tree.cl3d_phi[cl3d_index])**2)
        print('Tau index = %i - Cluster index = %i - Delta_R = %f' %(tau_index, cl3d_index, Delta_R_to_print))
    print('\n')

This is endcap2
This is the event number 1 corresponding to the entry 7302 with 1 tau(s) selected
Tau index = 3 - Cluster index = 0 - Delta_R = 0.017830


This is the event number 2 corresponding to the entry 7307 with 1 tau(s) selected
Tau index = 2 - Cluster index = 0 - Delta_R = 0.003380


This is the event number 4 corresponding to the entry 7312 with 1 tau(s) selected
Tau index = 3 - Cluster index = 1 - Delta_R = 0.068776


This is the event number 6 corresponding to the entry 7304 with 1 tau(s) selected
Tau index = 0 - Cluster index = 0 - Delta_R = 0.074759


This is the event number 7 corresponding to the entry 7306 with 1 tau(s) selected
Tau index = 0 - Cluster index = 0 - Delta_R = 0.025195


This is the event number 9 corresponding to the entry 7310 with 1 tau(s) selected
Tau index = 4 - Cluster index = 0 - Delta_R = 0.006098


This is the event number 10 corresponding to the entry 7308 with 3 tau(s) selected
Tau index = 0 - Cluster index = 0 - Delta_R = 0.016316
Tau index = 



This is the event number 326 corresponding to the entry 7750 with 2 tau(s) selected
Tau index = 0 - Cluster index = 2 - Delta_R = 0.003898
Tau index = 1 - Cluster index = 1 - Delta_R = 0.010043


This is the event number 327 corresponding to the entry 7743 with 1 tau(s) selected
Tau index = 1 - Cluster index = 0 - Delta_R = 0.008901


This is the event number 328 corresponding to the entry 7752 with 1 tau(s) selected
Tau index = 1 - Cluster index = 0 - Delta_R = 0.029790


This is the event number 329 corresponding to the entry 7745 with 3 tau(s) selected
Tau index = 0 - Cluster index = 0 - Delta_R = 0.016336
Tau index = 2 - Cluster index = 2 - Delta_R = 0.035135
Tau index = 4 - Cluster index = 1 - Delta_R = 0.020766


This is the event number 331 corresponding to the entry 7748 with 3 tau(s) selected
Tau index = 1 - Cluster index = 2 - Delta_R = 0.029362
Tau index = 2 - Cluster index = 1 - Delta_R = 0.020004
Tau index = 3 - Cluster index = 0 - Delta_R = 0.010833


This is the event 

Tau index = 2 - Cluster index = 2 - Delta_R = 0.070301


This is the event number 393 corresponding to the entry 7817 with 1 tau(s) selected
Tau index = 2 - Cluster index = 1 - Delta_R = 0.038085


This is the event number 395 corresponding to the entry 7819 with 2 tau(s) selected
Tau index = 0 - Cluster index = 2 - Delta_R = 0.016463
Tau index = 1 - Cluster index = 1 - Delta_R = 0.051444


This is the event number 399 corresponding to the entry 7828 with 2 tau(s) selected
Tau index = 0 - Cluster index = 1 - Delta_R = 0.023189
Tau index = 1 - Cluster index = 0 - Delta_R = 0.008565


This is the event number 400 corresponding to the entry 7814 with 2 tau(s) selected
Tau index = 1 - Cluster index = 1 - Delta_R = 0.009824
Tau index = 3 - Cluster index = 0 - Delta_R = 0.016773


This is the event number 401 corresponding to the entry 7821 with 2 tau(s) selected
Tau index = 1 - Cluster index = 1 - Delta_R = 0.036895
Tau index = 3 - Cluster index = 0 - Delta_R = 0.025684


This is the event 

Tau index = 0 - Cluster index = 1 - Delta_R = 0.043157
Tau index = 4 - Cluster index = 0 - Delta_R = 0.002076


This is the event number 602 corresponding to the entry 8047 with 1 tau(s) selected
Tau index = 1 - Cluster index = 1 - Delta_R = 0.081408


This is the event number 603 corresponding to the entry 8050 with 1 tau(s) selected
Tau index = 1 - Cluster index = 0 - Delta_R = 0.011610


This is the event number 604 corresponding to the entry 8048 with 3 tau(s) selected
Tau index = 1 - Cluster index = 0 - Delta_R = 0.007421
Tau index = 3 - Cluster index = 2 - Delta_R = 0.402709
Tau index = 6 - Cluster index = 1 - Delta_R = 0.008328


This is the event number 605 corresponding to the entry 8041 with 2 tau(s) selected
Tau index = 1 - Cluster index = 0 - Delta_R = 0.026985
Tau index = 2 - Cluster index = 1 - Delta_R = 0.007675


This is the event number 606 corresponding to the entry 8043 with 2 tau(s) selected
Tau index = 0 - Cluster index = 1 - Delta_R = 0.017656
Tau index = 1 - Clus

Tau index = 2 - Cluster index = 1 - Delta_R = 0.055177
Tau index = 4 - Cluster index = 3 - Delta_R = 0.024448


This is the event number 739 corresponding to the entry 8189 with 1 tau(s) selected
Tau index = 1 - Cluster index = 0 - Delta_R = 0.089326


This is the event number 740 corresponding to the entry 8196 with 2 tau(s) selected
Tau index = 0 - Cluster index = 2 - Delta_R = 0.018737
Tau index = 3 - Cluster index = 1 - Delta_R = 0.074267


This is the event number 741 corresponding to the entry 8195 with 2 tau(s) selected
Tau index = 0 - Cluster index = 1 - Delta_R = 0.020178
Tau index = 2 - Cluster index = 0 - Delta_R = 0.018897


This is the event number 743 corresponding to the entry 8197 with 1 tau(s) selected
Tau index = 2 - Cluster index = 0 - Delta_R = 0.044222


This is the event number 744 corresponding to the entry 8194 with 2 tau(s) selected
Tau index = 0 - Cluster index = 3 - Delta_R = 0.017831
Tau index = 2 - Cluster index = 4 - Delta_R = 0.022480


This is the event 

...

Tau index = 1 - Cluster index = 1 - Delta_R = 0.008688
Tau index = 3 - Cluster index = 0 - Delta_R = 0.053475
Tau index = 5 - Cluster index = 2 - Delta_R = 0.013137


This is the event number 7715 corresponding to the entry 1194 with 1 tau(s) selected
Tau index = 2 - Cluster index = 0 - Delta_R = 0.007020


This is the event number 7716 corresponding to the entry 1199 with 1 tau(s) selected
Tau index = 2 - Cluster index = 0 - Delta_R = 0.018043


This is the event number 7717 corresponding to the entry 1200 with 1 tau(s) selected
Tau index = 0 - Cluster index = 0 - Delta_R = 0.007327


This is the event number 7718 corresponding to the entry 1197 with 3 tau(s) selected
Tau index = 0 - Cluster index = 2 - Delta_R = 0.012541
Tau index = 2 - Cluster index = 1 - Delta_R = 0.016751
Tau index = 4 - Cluster index = 0 - Delta_R = 0.021323


This is the event number 7719 corresponding to the entry 1309 with 2 tau(s) selected
Tau index = 2 - Cluster index = 0 - Delta_R = 0.022303
Tau index = 4 -

Tau index = 1 - Cluster index = 0 - Delta_R = 0.024894


This is the event number 7930 corresponding to the entry 2822 with 2 tau(s) selected
Tau index = 1 - Cluster index = 1 - Delta_R = 0.108875
Tau index = 3 - Cluster index = 0 - Delta_R = 0.012907


This is the event number 7931 corresponding to the entry 2824 with 2 tau(s) selected
Tau index = 0 - Cluster index = 0 - Delta_R = 0.040471
Tau index = 2 - Cluster index = 1 - Delta_R = 0.012564


This is the event number 7932 corresponding to the entry 2820 with 1 tau(s) selected
Tau index = 1 - Cluster index = 1 - Delta_R = 0.009130


This is the event number 7933 corresponding to the entry 2816 with 2 tau(s) selected
Tau index = 2 - Cluster index = 0 - Delta_R = 0.013658
Tau index = 4 - Cluster index = 2 - Delta_R = 0.018263


This is the event number 7935 corresponding to the entry 2825 with 2 tau(s) selected
Tau index = 2 - Cluster index = 1 - Delta_R = 0.012343
Tau index = 3 - Cluster index = 0 - Delta_R = 0.866403


This is the e

This is the event number 8067 corresponding to the entry 3169 with 1 tau(s) selected
Tau index = 3 - Cluster index = 0 - Delta_R = 0.016709


This is the event number 8068 corresponding to the entry 3177 with 1 tau(s) selected
Tau index = 1 - Cluster index = 0 - Delta_R = 0.046164


This is the event number 8069 corresponding to the entry 3173 with 2 tau(s) selected
Tau index = 1 - Cluster index = 1 - Delta_R = 0.006546
Tau index = 3 - Cluster index = 0 - Delta_R = 0.057891


This is the event number 8070 corresponding to the entry 3174 with 1 tau(s) selected
Tau index = 2 - Cluster index = 1 - Delta_R = 0.749259


This is the event number 8071 corresponding to the entry 3172 with 3 tau(s) selected
Tau index = 0 - Cluster index = 1 - Delta_R = 0.020482
Tau index = 2 - Cluster index = 2 - Delta_R = 0.032331
Tau index = 3 - Cluster index = 0 - Delta_R = 0.010225


This is the event number 8072 corresponding to the entry 3182 with 1 tau(s) selected
Tau index = 2 - Cluster index = 0 - Delt

Tau index = 3 - Cluster index = 1 - Delta_R = 0.005899


This is the event number 8274 corresponding to the entry 3492 with 1 tau(s) selected
Tau index = 0 - Cluster index = 0 - Delta_R = 0.824991


This is the event number 8276 corresponding to the entry 3489 with 2 tau(s) selected
Tau index = 0 - Cluster index = 1 - Delta_R = 0.349521
Tau index = 4 - Cluster index = 0 - Delta_R = 0.011993


This is the event number 8277 corresponding to the entry 3491 with 1 tau(s) selected
Tau index = 1 - Cluster index = 0 - Delta_R = 0.012661


This is the event number 8279 corresponding to the entry 3490 with 2 tau(s) selected
Tau index = 1 - Cluster index = 1 - Delta_R = 0.006106
Tau index = 3 - Cluster index = 0 - Delta_R = 0.073505


This is the event number 8280 corresponding to the entry 3493 with 1 tau(s) selected
Tau index = 2 - Cluster index = 0 - Delta_R = 0.041299


This is the event number 8281 corresponding to the entry 3496 with 1 tau(s) selected
Tau index = 0 - Cluster index = 1 - De

This is the event number 8410 corresponding to the entry 3639 with 1 tau(s) selected
Tau index = 0 - Cluster index = 1 - Delta_R = 0.038456


This is the event number 8411 corresponding to the entry 3630 with 1 tau(s) selected
Tau index = 0 - Cluster index = 1 - Delta_R = 0.009673


This is the event number 8413 corresponding to the entry 3631 with 1 tau(s) selected
Tau index = 3 - Cluster index = 1 - Delta_R = 0.027734


This is the event number 8414 corresponding to the entry 3634 with 1 tau(s) selected
Tau index = 2 - Cluster index = 2 - Delta_R = 0.015804


This is the event number 8416 corresponding to the entry 3640 with 2 tau(s) selected
Tau index = 0 - Cluster index = 2 - Delta_R = 0.036482
Tau index = 1 - Cluster index = 1 - Delta_R = 0.628931


This is the event number 8417 corresponding to the entry 3638 with 2 tau(s) selected
Tau index = 2 - Cluster index = 1 - Delta_R = 0.017901
Tau index = 5 - Cluster index = 2 - Delta_R = 0.004399


This is the event number 8418 correspo